# Ownership - Version 1

In [ ]:
%%capture

import pandas as pd
import plotly.express as px
import aesoppy.aesop as aesop
import os

# Comon Variables
data = os.path.join(os.getcwd(), 'data')

# DataFrames
fin_df = pd.read_csv(os.path.join(data, 'financials.csv'), index_col='fiscal_year')
owners_df = aesop.Owners(financials_df=fin_df).owners_df

In [ ]:
# Reduce Dataframe Index By year
for index, row in owners_df.iterrows():
    if aesop.aesop_now.year - index >= 21:
        owners_df.drop(index, inplace=True)

owners_df

## Share Growth/Decrease

In [ ]:
share_df = pd.DataFrame(owners_df[['shares_outstanding_eop', 'shares_outstanding_diluted']], index=owners_df.index)
share_df['sharegro_eop'] = share_df['shares_outstanding_eop'].pct_change()
share_df['sharegro_eop'] = round(share_df['sharegro_eop'] * 100, 2)
share_df['sharegro_diluted'] = share_df['shares_outstanding_diluted'].pct_change()
share_df['sharegro_diluted'] = round(share_df['sharegro_diluted'] * 100, 2)
share_df

In [ ]:
share_fig1 = px.line(share_df, x=share_df.index, y=['shares_outstanding_eop', 'shares_outstanding_diluted'], color_discrete_sequence=['blue', 'yellow'])
share_fig1.show()

## Owners Return

In [ ]:
return_df = pd.DataFrame(owners_df[['cash_from_ops', 'cash_for_dividends', 'stock_buyback', 'stock_issues']], index=owners_df.index)
return_df['stock_total'] = return_df['stock_buyback'] + return_df['stock_issues']
return_df['owners_total'] = (return_df['cash_for_dividends'] + return_df['stock_total']).abs()
return_df['owners_return'] = round((return_df['owners_total'] / return_df['cash_from_ops']) * 100, 2)
return_df['buyback_total'] = return_df['stock_total'].abs()
return_df['dividends_paid'] = return_df['cash_for_dividends'].abs()

return_df

In [ ]:
fig1 = px.bar(return_df, x=return_df.index, y=['buyback_total', 'dividends_paid', 'cash_from_ops'],
              barmode='group', color_discrete_sequence=['blue', 'yellow', 'green'])
fig1.show()